# Generate air traffic heatmap from so6 files

In [1]:
import pandas as pd
from datetime import datetime
import glob
from tqdm import tqdm
from ipyleaflet import Map, Heatmap, basemaps, basemap_to_tiles, Marker, MarkerCluster
from ipywidgets import Layout
from ipywidgets.embed import embed_minimal_html

In [2]:
default_zoom = 5
default_radius = 10
france_barycenter = (46.45, 2.49)

#### <br> Generate a pandas dataframe from a so6 file

In [3]:
def so6_to_df(path):
    columns = ['segment_identifier', 'flight_origin', 'flight_destination', 'aircraft_type', 
           'time_begin', 'time_end', 'fl_begin', 'fl_end', 'status', 'callsign', 
           'date_begin', 'date_end', 'lat_begin', 'lon_begin', 'lat_end', 'lon_end',
           'flight_identifier', 'sequence', 'length', 'parity']
    parser = lambda x,y: pd.datetime.strptime(x + y, '%y%m%d%H%M%S').replace(second=0)

    df = pd.read_csv(path, sep=' ', header=None, names=columns, 
                     parse_dates=[['date_begin','time_begin'],['date_end','time_end']], date_parser=parser)

    df.query('length > 0', inplace=True) # filter null segments
    df['duration'] = (df['date_end_time_end'] - df['date_begin_time_begin']).dt.total_seconds()
    df['alt_begin (m)'] = df['fl_begin'].apply(lambda x: x * 30.48) # FL to meters
    df['alt_end (m)'] = df['fl_end'].apply(lambda x: x * 30.48) # FL to meters
    lat_lon_cols = ['lat_begin','lon_begin','lat_end','lon_end']
    df[lat_lon_cols] = df[lat_lon_cols].apply(lambda x: x / 60)
    df = df[['date_begin_time_begin', 'lat_begin', 'lon_begin', 'flight_identifier']]
    df.set_index('date_begin_time_begin', inplace=True)
    return df

#### <br> Merge all tables (so6 files)

In [4]:
df_list = []
for file in tqdm(glob.glob('so6_files/*.so6')): # your folder with so6 files
    df_list.append(so6_to_df(file))
t_df = pd.concat(df_list)
print('Table generated with {} lines'.format(t_df['flight_identifier'].count()))

100%|██████████| 1/1 [00:43<00:00, 43.65s/it]

Table generated with 1524772 lines


#### <br> Get points from trajectories after resampling (every minute)

In [5]:
locations = []
for name, group in t_df.groupby('flight_identifier'):
    group = group[~group.index.duplicated(keep='first')] # keep only the first point if several found in the same minute
    i_group = group.resample('T').asfreq().interpolate()
    locations.extend(list(zip(i_group.lat_begin, i_group.lon_begin)))

#### <br> Generate heatmap with ipyleaflet

In [8]:
terrain = basemap_to_tiles(basemaps.Stamen.Terrain)
m = Map(layers=(terrain, ), center=france_barycenter, zoom=default_zoom, layout=Layout(width='100%', height='600px'))
heatmap = Heatmap(locations=locations, radius=default_radius)
m.add_layer(heatmap)

#### <br> Export the map in HTML

In [7]:
embed_minimal_html('heatmap.html', views=[m], title='air traffic heatmap')